In [2]:
import matplotlib
matplotlib.use('agg')

In [3]:
%matplotlib inline

In [4]:
import phenom
import os
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from importlib import reload

reload(phenom.dataset)
reload(phenom.phenotype)

<module 'phenom.phenotype' from '/home/ubuntu/phenom/phenom/phenotype.py'>

# low oxidative 

In [6]:
ds = None

for dr in os.listdir("data/low-oxidative/"):
    
    if not os.path.isdir(os.path.join("data/low-oxidative/", dr)):
        continue
    
    if ds is None:
        ds = phenom.dataset.DataSet.fromDirectory(os.path.join("data/low-oxidative", dr))
    else:
        ds = ds.concat(phenom.dataset.DataSet.fromDirectory(os.path.join("data/low-oxidative", dr)))
        
ds.filter()
ds.meta["mMPQ"] = ds.meta["mM PQ"]

In [7]:
mnull = phenom.design.Formula(ds.meta, "C(mMPQ, Sum)")

hierarchy = phenom.design.Formula(ds.meta, "1") + phenom.design.Formula(ds.meta, "C(plate) + 0")
mbatch = mnull * hierarchy

In [8]:
null_phen = phenom.phenotype.Phenotype(ds.data, mnull, model="phenom_deriv.stan")

batch_phen = phenom.phenotype.Phenotype(ds.data, mbatch, model="phenom_deriv.stan",
                                        lengthscale_priors=[[6, 1], [6, 1], [6, 1], [6, 1]], 
                                       alpha_priors= [[10.0, 10.0], [10.0, 10.0], [7.0, 10.0], [7.0, 10.0]],
                                       minExpectedCross=.1, maxExpectedCross=3,
                                       sigma_prior=[.02, 20])

full_phen = phenom.phenotype.Phenotype(ds.data, 
                                       mbatch, model="phenom_marginal.stan", 
                                       lengthscale_priors=[[6, 1], [6, 1], [6, 1], [6, 1]], 
                                       alpha_priors= [[10.0, 10.0], [10.0, 10.0], [7.0, 10.0], [7.0, 10.0]],
                                       marginal_lengthscale_prior=[6, 1],
                                       marginal_alpha_prior=[2, 100],
                                       minExpectedCross=.1, maxExpectedCross=3,
                                       sigma_prior=[.02, 20]
                                      )

In [10]:
null_samples = null_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


In [11]:
print(null_samples)

Inference for Stan model: anon_model_7226d8af40a102760f76e1d6f960daef.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]    0.5  2.0e-3   0.08   0.35   0.44    0.5   0.55   0.66   1516    1.0
lengthscale[2]   0.52  8.0e-3   0.28   0.25   0.36   0.45    0.6   1.25   1206    1.0
alpha[1]         1.32    0.01   0.47   0.64   0.96   1.23   1.59   2.45   1141    1.0
alpha[2]         0.17  5.6e-3   0.21   0.04   0.07   0.11   0.18   0.71   1383    1.0
sigma            0.45  4.8e-5 3.9e-3   0.44   0.45   0.45   0.45   0.46   6746    1.0
f_eta[1,1]      -1.38    0.01   0.48   -2.5  -1.67  -1.31  -1.01  -0.66   1594    1.0
f_eta[2,1]      -0.12  5.2e-3    0.2  -0.59  -0.22  -0.09-8.0e-3   0.27   1434    1.0
f_eta[1,2]       0.81  6.6e-3   0.28   0.35    0.6   0.78   0.97   1.49   1880    1.0
f_eta[2,2]       0.45    0.02   0.71 

In [12]:
null_phen.save("samples/hsalinarum/low-oxidative/null")

In [ ]:
batch_samples = batch_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


In [12]:
print(batch_samples)

Inference for Stan model: anon_model_7226d8af40a102760f76e1d6f960daef.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]   0.43  1.7e-3   0.06   0.31   0.39   0.43   0.48   0.56   1388    1.0
lengthscale[2]   0.44  2.1e-3   0.09   0.28   0.38   0.44    0.5   0.61   1703    1.0
lengthscale[3]   0.32  5.8e-4   0.02   0.28   0.31   0.32   0.34   0.36   1345    1.0
lengthscale[4]   0.41  2.5e-3   0.06    0.3   0.37    0.4   0.45   0.56    687   1.01
alpha[1]         1.02  5.1e-3   0.24   0.63   0.85   0.99   1.16   1.55   2220    1.0
alpha[2]         0.61  5.3e-3   0.24   0.24   0.44   0.58   0.75   1.17   2107    1.0
alpha[3]         0.32  1.2e-3   0.05   0.25   0.29   0.32   0.35   0.43   1494    1.0
alpha[4]         0.13  7.6e-4   0.02    0.1   0.12   0.13   0.15   0.19    954   1.01
sigma            0.18  1.8e-5 1.6e-3 

In [13]:
batch_phen.save("samples/hsalinarum/low-oxidative/batch")

In [ ]:
full_samples = full_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1952fe1216b95ba3e2733fb643947f40 NOW.


In [15]:
print(full_samples)

Inference for Stan model: anon_model_1952fe1216b95ba3e2733fb643947f40.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]         0.44  2.9e-3   0.11   0.26   0.36   0.43   0.51   0.69   1472    1.0
lengthscale[2]         0.56  7.8e-3   0.23   0.28   0.41    0.5   0.63   1.18    886   1.01
lengthscale[3]         0.11  1.3e-5 8.0e-4   0.11   0.11   0.11   0.11   0.11   3735    1.0
lengthscale[4]         0.16  1.8e-4 6.3e-3   0.15   0.16   0.16   0.17   0.17   1269    1.0
alpha[1]               1.05  4.6e-3   0.25   0.65   0.88   1.02   1.19   1.65   2946    1.0
alpha[2]                0.6  4.2e-3   0.24   0.23   0.43   0.57   0.75   1.14   3215    1.0
alpha[3]               0.28  6.8e-4   0.02   0.24   0.27   0.28    0.3   0.33   1168    1.0
alpha[4]                0.1  3.0e-4   0.01   0.08   0.09    0.1   0.11   0.

In [16]:
full_phen.save("samples/hsalinarum/low-oxidative/full")

# high oxidative 

In [14]:
ds = None

for dr in os.listdir("data/hi-oxidative/"):
    
    if not os.path.isdir(os.path.join("data/hi-oxidative/", dr)):
        continue
    
    if ds is None:
        ds = phenom.dataset.DataSet.fromDirectory(os.path.join("data/hi-oxidative", dr))
    else:
        ds = ds.concat(phenom.dataset.DataSet.fromDirectory(os.path.join("data/hi-oxidative", dr)))
        
ds.filter()
ds.meta["mMPQ"] = ds.meta["mM PQ"]

In [16]:
mnull = phenom.design.Formula(ds.meta, "C(mMPQ, Sum)")

hierarchy = phenom.design.Formula(ds.meta, "1") + phenom.design.Formula(ds.meta, "C(plate) + 0")
mbatch = mnull * hierarchy

In [19]:
null_phen = phenom.phenotype.Phenotype(ds.data, mnull, model="phenom_deriv.stan")
batch_phen = phenom.phenotype.Phenotype(ds.data, mbatch, model="phenom_deriv.stan",
                                       lengthscale_priors=[[6, 1], [6, 1], [6, 1], [6, 1]], 
                                       alpha_priors= [[10.0, 10.0], [10.0, 10.0], [7.0, 10.0], [7.0, 10.0]],
                                       minExpectedCross=.1, maxExpectedCross=3,
                                       sigma_prior=[.02, 20])

In [20]:
null_samples = null_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


In [21]:
print(null_samples)

Inference for Stan model: anon_model_7226d8af40a102760f76e1d6f960daef.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]   0.51  1.8e-3   0.08   0.35   0.46   0.51   0.56   0.66   1827    1.0
lengthscale[2]   0.67    0.02   0.43   0.29    0.4   0.52   0.81   1.84    583   1.01
alpha[1]         1.22    0.01   0.47    0.6   0.89   1.11   1.47   2.36   1505    1.0
alpha[2]         0.68  9.0e-3    0.4   0.26   0.43   0.58   0.82   1.75   1978    1.0
sigma            0.46  4.4e-5 3.9e-3   0.46   0.46   0.46   0.47   0.47   7567    1.0
f_eta[1,1]      -1.36    0.01   0.48  -2.44  -1.66  -1.31   -1.0  -0.62   1474    1.0
f_eta[2,1]      -0.04  1.8e-3   0.06  -0.17  -0.07  -0.04-7.0e-3   0.07   1077    1.0
f_eta[1,2]       0.96  7.7e-3   0.32   0.46   0.72   0.92   1.15   1.66   1705    1.0
f_eta[2,2]       0.39    0.03   0.65 

In [22]:
null_phen.save("samples/hsalinarum/hi-oxidative/null")

In [ ]:
batch_samples = batch_phen.samples()

In [ ]:
batch_phen.save("samples/hsalinarum/hi-oxidative/batch")

In [20]:
print(batch_samples)

Inference for Stan model: anon_model_7226d8af40a102760f76e1d6f960daef.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]   0.46  2.6e-3   0.07   0.34   0.42   0.46   0.51   0.61    665   1.01
lengthscale[2]   0.62    0.05   0.58   0.25    0.3   0.38   0.75   2.21    150   1.02
lengthscale[3]   0.32  8.1e-4   0.02   0.28   0.31   0.32   0.34   0.37    657    1.0
lengthscale[4]   0.22  2.4e-3   0.03   0.18   0.21   0.22   0.24   0.28    110   1.02
alpha[1]         1.49    0.02   0.58    0.7   1.07   1.38   1.79   3.02    563   1.01
alpha[2]         1.07    0.03   0.58   0.47    0.7   0.92   1.25   2.53    349   1.01
alpha[3]         0.39  2.0e-3   0.06   0.31   0.36   0.39   0.42   0.52    773    1.0
alpha[4]         0.24  1.1e-3   0.03   0.19   0.22   0.24   0.26   0.31    818    1.0
sigma            0.19  3.1e-5 1.7e-3 

In [ ]:
full_phen = phenom.phenotype.Phenotype(ds.data, 
                                       mbatch, model="phenom_marginal.stan", 
                                       lengthscale_priors=[[6, 1], [6, 1], [6, 1], [6, 1]], 
                                       alpha_priors= [[10.0, 10.0], [10.0, 10.0], [7.0, 10.0], [7.0, 10.0]],
                                       marginal_lengthscale_prior=[6, 1],
                                       marginal_alpha_prior=[2, 100],
                                       minExpectedCross=.1, maxExpectedCross=3,
                                       sigma_prior=[.02, 20]
                                      )

In [ ]:
full_samples = full_phen.samples()

In [26]:
print(full_samples)

Inference for Stan model: anon_model_1952fe1216b95ba3e2733fb643947f40.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]         0.42  3.1e-3   0.11   0.24   0.34   0.41   0.48   0.66   1188    1.0
lengthscale[2]         0.39  4.8e-3   0.11   0.23   0.32   0.38   0.44   0.67    509    1.0
lengthscale[3]         0.11  8.7e-6 5.8e-4   0.11   0.11   0.11   0.11   0.11   4501    1.0
lengthscale[4]         0.12  6.7e-4 9.1e-3   0.11   0.11   0.12   0.13   0.14    185   1.01
alpha[1]                1.1  4.5e-3   0.24   0.71   0.93   1.08   1.25   1.63   2758    1.0
alpha[2]               1.04  4.3e-3   0.26   0.64   0.86   1.01   1.19   1.64   3642    1.0
alpha[3]               0.38  1.2e-3   0.03   0.32   0.36   0.38   0.41   0.46    879    1.0
alpha[4]               0.22  6.1e-4   0.02   0.19   0.21   0.22   0.24   0.

In [ ]:
full_phen.save("samples/hsalinarum/hi-oxidative/full")